In [3]:
# !pip3 install grpcio==1.58.0 grpcio-tools==1.58.0 --break-system-packages

In [30]:
protobuf = '''
syntax = "proto3";

// messages (protocol buffers, aka "protobufs")

message MultReq {
    int32 x = 1;
    int32 y = 2;
}

message MultResp { 
    int32 result = 1;
}

// services (RPC functions)

service Calc {
    rpc Mult(MultReq) returns (MultResp);
}
'''
with open("math.proto", "w") as f:
    f.write(protobuf)

In [31]:
# !cat math.proto

In [32]:
# !python3 -m grpc_tools.protoc -I=. --python_out=. --grpc_python_out=. math.proto

In [28]:
# !ls -lah

In [20]:
# ! cat math_pb2_grpc.py

In [35]:
import math_pb2

In [51]:
pb = math_pb2.MultReq(x=16, y=32)
pb

x: 16
y: 32

In [52]:
s = pb.SerializePartialToString()
s, type(s), len(s)

(b'\x08\x10\x10 ', bytes, 4)

In [47]:
pb = math_pb2.MultReq(x=3, y=40000)
s = pb.SerializePartialToString()
s, len(s)

(b'\x08\x03\x10\xc0\xb8\x02', 6)

In [50]:
pb = math_pb2.MultReq(x=3, y=42432452342346234)
s = pb.SerializePartialToString()
s, len(s)

ValueError: Value out of range: 42432452342346234

In [33]:
server = '''
import grpc
import math_pb2_grpc, math_pb2
from concurrent import futures

class MyCalc(CalcServicer):
    def Mult(self, request, context):
        print("hi")
        print(request)
        # return request.x * request.y
        return math_pb2.MultResp(result = request.x * request.y)

server = grpc.server(futures.ThreadPoolExecutor(max_workers=10), 
                        options=[("grpc.so_reuseport", 0)])

math_pb2_grpc.add_CalcServicer_to_server(MyCalc(), server)
server.add_insecure_port('0.0.0.0:5440')
server.start()
print("started")
server.wait_for_termination()
'''

with open("server.py", "w") as f:
    f.write(server)

In [34]:
!python3 server.py

Traceback (most recent call last):
  File "/nb/server.py", line 6, in <module>
    class MyCalc(CalcServicer):
                 ^^^^^^^^^^^^
NameError: name 'CalcServicer' is not defined
